In [1]:
import sys
sys.path.insert(0, '/home/mint/Relighting_preprocessing/Relighting_preprocessing_tools/DECA/decalib')
from datasets.datasets import TestData_Array

In [2]:
import numpy as np
import blobfile as bf
import PIL
import matplotlib.pyplot as plt
import tqdm

def _list_image_files_recursively(data_dir):
    results = []
    for entry in sorted(bf.listdir(data_dir)):
        full_path = bf.join(data_dir, entry)
        ext = entry.split(".")[-1]
        if "." in entry and ext.lower() in ["jpg", "jpeg", "png", "gif", "npy"]:
            results.append(full_path)
        elif bf.isdir(full_path):
            results.extend(_list_image_files_recursively(full_path))
    return results

def load_image(path):
    with bf.BlobFile(path, "rb") as f:
        pil_image = PIL.Image.open(f)
        pil_image.load()
    pil_image = pil_image.convert("RGB")
    return pil_image

def resize_arr(pil_image, image_size):
    img = pil_image.resize((image_size, image_size), PIL.Image.ANTIALIAS)
    return img

def rgb_to_gray(img):
    out = (img[..., [0]] * 0.2989) + (img[..., [1]] * 0.5870) + (img[..., [2]] * 0.1140)
    return out




In [31]:
set_ = 'train'
img_path = _list_image_files_recursively(f'/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/{set_}/')
img = np.array(load_image(img_path[0]))
fo_rgb = open(f"./ffhq-{set_}-norm-rgb-anno.txt", "w")
fo_gray = open(f"./ffhq-{set_}-norm-gray-anno.txt", "w")

for i in tqdm.tqdm(range(len(img_path))):
    name = img_path[i].split('/')[-1]
    img = np.array(load_image(img_path[i]))
    img_gray = rgb_to_gray(img)
    
    mu_rgb, std_rgb = np.mean(img), np.std(img)
    mu_gray, std_gray = np.mean(img_gray), np.std(img_gray)
    fo_rgb.write(name + " ")
    fo_rgb.write(" ".join([str(x) for x in [mu_rgb, std_rgb]]) + "\n")
    
    fo_gray.write(name + " ")
    fo_gray.write(" ".join([str(x) for x in [mu_gray, std_gray]]) + "\n")

fo_rgb.close()
fo_gray.close()

100%|██████████| 60000/60000 [08:13<00:00, 121.68it/s]


In [4]:
set_ = 'train'
img_path = _list_image_files_recursively(f'/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/{set_}/')
img = np.array(load_image(img_path[0]))
img_gray = rgb_to_gray(img)

test_dat = TestData_Array(test_dat={img_path[0].split('/')[-1] : img})

mu_rgb, std_rgb = np.mean(img), np.std(img)
print(np.max((img - mu_rgb)/std_rgb), np.min(img))

2.278922221909528 0


In [36]:
TestData_Array(test_dat={img_path[0].split('/')[-1]} : img)

dict_values([3, 5])